In [3]:
import config
import geocoder
import pandas as pd
import folium

In [4]:
postal_codes_MID = pd.read_excel(r"C:\Users\william.gruintal\Desktop\coursera_projects\Coursera_Capstone_Final\CodigosPostalesMerida_20210517.xlsx")
headers = postal_codes_MID.iloc[0]
postal_codes_MID = postal_codes_MID.iloc[1:]
postal_codes_MID.columns = headers
postal_codes_MID.columns = ['CodigoPostal', 'Estado', 'Municipio', 'Ciudad', 'Tipo', 'Asentamiento', 'Oficina']
postal_codes_MID.drop(['Oficina', 'Estado', 'Municipio', 'Ciudad'], axis = 1, inplace = True)
postal_codes_MID.reset_index(drop = True, inplace = True)
postal_codes_MID.head(30)

,CodigoPostal,Tipo,Asentamiento
0,97000,Colonia,Mérida Centro
1,97000,Fraccionamiento,La Quinta
2,97000,Fraccionamiento,Los Cocos
3,97000,Fraccionamiento,Privada Del Maestro
4,97000,Colonia,Privada Garcia Gineres C - 29
5,97000,Fraccionamiento,Jardines de San Sebastian
6,97003,Fraccionamiento,Los Reyes
7,97050,Colonia,Alcalá Martín
8,97050,Colonia,Yucatán
9,97059,Fraccionamiento,Señorial


In [5]:
postal_codes_MID.describe()

,CodigoPostal,Tipo,Asentamiento
count,632,632,632
unique,140,16,619
top,97203,Fraccionamiento,Vista Alegre
freq,35,383,2


In [6]:
print('postal_codes_MID dataframe shape is: {} rows and {} columns'.format(postal_codes_MID.shape[0], postal_codes_MID.shape[1]))

# check for null values
print ('Nomber of null postal codes: ', postal_codes_MID['CodigoPostal'].isnull().sum())

#check for null values
print('Number of unique postal codes: ', len(postal_codes_MID.CodigoPostal.unique()))
print('Number of unique neighborhoods: ', len(postal_codes_MID.Asentamiento.unique()))
print('Number of duplicated values in column "Asentamiento" (neighborhood):', len(postal_codes_MID[postal_codes_MID.duplicated(['Asentamiento'])]))

postal_codes_MID dataframe shape is: 632 rows and 3 columns
Nomber of null postal codes:  0
Number of unique postal codes:  140
Number of unique neighborhoods:  619
Number of duplicated values in column "Asentamiento" (neighborhood): 13


In [7]:
postal_codes_MID[postal_codes_MID.duplicated(['Asentamiento'], keep = 'first')]

,CodigoPostal,Tipo,Asentamiento
22,97100,Fraccionamiento,Itzimna
55,97119,Colonia,Gonzalo Guerrero
78,97130,Fraccionamiento,Vista Alegre
144,97156,Fraccionamiento,Los Reyes
152,97159,Colonia,Nueva Mayapan
156,97160,Fraccionamiento,Manuel Avila Camacho
202,97195,Colonia,San Antonio Kaua
402,97249,Colonia,Mulsay
419,97249,Colonia,Hacienda Opichen
462,97284,Fraccionamiento,Lindavista


In [8]:
#testing
#postal_codes_MID = postal_codes_MID.iloc[0:150]
#postal_codes_MID

In [9]:
#postal_codes = [97179, 97000]
lat_long_coords = []
for cp, type_, neigh in zip (postal_codes_MID['CodigoPostal'], postal_codes_MID['Tipo'], postal_codes_MID['Asentamiento']):
    lat_long1 = [cp, type_, neigh]
    lat_long2 = geocoder.arcgis('{},{},{}, Merida, YUC'.format(cp, type_, neigh))
    while(lat_long2.latlng is None):
        lat_long = geocoder.arcgis('{},{},{}, Merida, YUC'.format(cp, type_, neigh))
        print (type_, neigh, lat_long2.latlng)
    lat_long1.extend(lat_long2.latlng)
    lat_long_coords.append(lat_long1)
    if len(lat_long_coords) / postal_codes_MID['CodigoPostal'].shape[0] == 0.25:
        print('progress: 25%')
    if len(lat_long_coords) / postal_codes_MID['CodigoPostal'].shape[0] == 0.50:
        print('progress: 50%')
    if len(lat_long_coords) / postal_codes_MID['CodigoPostal'].shape[0] == 0.75:
        print('progress: 75%')
    if len(lat_long_coords) / postal_codes_MID['CodigoPostal'].shape[0] == 1:
        print('progress: 100%')
#    print(neigh, lat_long2.latlng)
print('Coordinates loaded')

progress: 25%
progress: 50%
progress: 75%
progress: 100%
Coordinates loaded


In [10]:
columns = ['CodigoPostal', 'Tipo', 'Neighborhood', 'Latitud', 'Longitud']
lat_long_coords = pd.DataFrame(lat_long_coords)
lat_long_coords.columns = columns
lat_long_coords.head(30)

,CodigoPostal,Tipo,Neighborhood,Latitud,Longitud
0,97000,Colonia,Mérida Centro,20.961655,-89.630230
1,97000,Fraccionamiento,La Quinta,20.982700,-89.613480
2,97000,Fraccionamiento,Los Cocos,21.007190,-89.569840
3,97000,Fraccionamiento,Privada Del Maestro,20.995640,-89.640300
4,97000,Colonia,Privada Garcia Gineres C - 29,20.984409,-89.633806
5,97000,Fraccionamiento,Jardines de San Sebastian,20.958369,-89.633441
6,97003,Fraccionamiento,Los Reyes,20.951099,-89.643857
7,97050,Colonia,Alcalá Martín,20.986802,-89.618844
8,97050,Colonia,Yucatán,20.996148,-89.625719
9,97059,Fraccionamiento,Señorial,20.994450,-89.595410


In [11]:
lat_long_coords.shape

(632, 5)

In [12]:
print('Number of unique types ("Tipo"): ', len(lat_long_coords.Tipo.unique()))
print(lat_long_coords.Tipo.unique())
print('Number of unique neighborhoods: ', len(lat_long_coords.Neighborhood.unique()))

Number of unique types ("Tipo"):  16
['Colonia' 'Fraccionamiento' 'Unidad habitacional' 'Residencial'
 'Gran usuario' 'Zona comercial' 'Conjunto habitacional' 'Zona industrial'
 'Ampliación' 'Aeropuerto' 'Pueblo' 'Hacienda' 'Ranchería' 'Condominio'
 'Rancho' 'Equipamiento']
Number of unique neighborhoods:  619


In [13]:
#print('• postal_codes_MID data types:')
#print(postal_codes_MID.dtypes)
#print()
print('• lat_long_coords data types')
print(lat_long_coords.dtypes)

• lat_long_coords data types
CodigoPostal      int64
Tipo             object
Neighborhood     object
Latitud         float64
Longitud        float64
dtype: object


In [14]:
lat_long_coords['CodigoPostal'] = lat_long_coords['CodigoPostal'].astype('str')
merida_neighborhoods = lat_long_coords
print('• merida_neighborhoods data types:')
print(lat_long_coords.dtypes)
merida_neighborhoods.head(30)

• merida_neighborhoods data types:
CodigoPostal     object
Tipo             object
Neighborhood     object
Latitud         float64
Longitud        float64
dtype: object


,CodigoPostal,Tipo,Neighborhood,Latitud,Longitud
0,97000,Colonia,Mérida Centro,20.961655,-89.630230
1,97000,Fraccionamiento,La Quinta,20.982700,-89.613480
2,97000,Fraccionamiento,Los Cocos,21.007190,-89.569840
3,97000,Fraccionamiento,Privada Del Maestro,20.995640,-89.640300
4,97000,Colonia,Privada Garcia Gineres C - 29,20.984409,-89.633806
5,97000,Fraccionamiento,Jardines de San Sebastian,20.958369,-89.633441
6,97003,Fraccionamiento,Los Reyes,20.951099,-89.643857
7,97050,Colonia,Alcalá Martín,20.986802,-89.618844
8,97050,Colonia,Yucatán,20.996148,-89.625719
9,97059,Fraccionamiento,Señorial,20.994450,-89.595410


In [15]:
merida_neighborhoods.shape

(632, 5)

In [16]:
#postal_codes_MID.rename(columns ={'Asentamiento': 'Neighborhood'}, inplace = True)
#print(postal_codes_MID.columns)
#print(lat_long_coords.columns)

In [17]:
# merge dataframes
#merida_neighborhoods = pd.merge(left = postal_codes_MID, right = lat_long_coords, left_on = ['CodigoPostal', 'Tipo', 'Neighborhood'], right_on = ['CodigoPostal', 'Tipo', 'Neighborhood'])
#merida_neighborhoods.head(30)

In [18]:
#merida_neighborhoods.shape

In [19]:
print('Number of unique postal codes: ', len(merida_neighborhoods.CodigoPostal.unique()))
print('Number of unique neighborhoods: ', len(merida_neighborhoods.Neighborhood.unique()))

Number of unique postal codes:  140
Number of unique neighborhoods:  619


In [20]:
g = geocoder.arcgis('Merida, YUC')
latitude = g.latlng[0]
longitude = g.latlng[1]
print('Mérida, Yucatán coordinates are: {}, {}.'.format(g.latlng[0], g.latlng[1]))

Mérida, Yucatán coordinates are: 20.96670000000006, -89.62316999999996.


In [21]:
map_merida = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, tipo, neighborhood in zip(merida_neighborhoods['Latitud'], merida_neighborhoods['Longitud'], merida_neighborhoods['Tipo'], merida_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(tipo,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_merida)  
    
map_merida